In [2]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier

from preprocess import extract_features

## Checking the individual models

In [14]:

def get_predictions(model):
    unconfident_inputs = extract_features("../../samples/unconfident.m4a")
    confident_inputs = extract_features("../../samples/confident.m4a")
    tim_urban_inputs = extract_features("../../samples/tim-urban.m4a")
    almost_silent_inputs = extract_features('../../samples/almost silent.m4a')
    stutter_inputs = extract_features("../../samples/stutter.m4a")
    
    speeches = ['Unconfident speech', "Confident speech", "Tim urban's speech", "Almost silent speech", "Stuttering speech"]
    classes = ["Low", "Medium", "High"]
    
    inputs = np.array([confident_inputs, unconfident_inputs, tim_urban_inputs, almost_silent_inputs, stutter_inputs])
    
    try:
        res = model.predict_proba(inputs)
    except:
        res = model.predict(inputs)
    
    for i in range(len(res)):
        print(f"{speeches[i]} has fluency prediction {classes[res[i].argmax()]} with max probability {res[i].max()} and all probabilities {res[i]}")
        
    return res

In [15]:
# Models
lgbm = joblib.load("weights/lightgbm_model.pkl", "r")
xgbm = joblib.load("weights/xgboost_model.pkl", "r")
rf = joblib.load("weights/random_forest_model.pkl", "r")
stacking = joblib.load("weights/stacking_model.pkl", "r")
voting = joblib.load("weights/voting_model.pkl", "r")

In [16]:
print("LGBM:")
lgbm_preds = get_predictions(lgbm)

LGBM:
Unconfident speech has fluency prediction High with max probability 0.7624521991316683 and all probabilities [0.11008127 0.12746653 0.7624522 ]
Confident speech has fluency prediction High with max probability 0.8889332781983281 and all probabilities [0.04624357 0.06482315 0.88893328]
Tim urban's speech has fluency prediction Low with max probability 0.5804870330573566 and all probabilities [0.58048703 0.38668391 0.03282905]
Almost silent speech has fluency prediction High with max probability 0.8386745552642046 and all probabilities [0.10985609 0.05146936 0.83867456]
Stuttering speech has fluency prediction High with max probability 0.773176976869852 and all probabilities [0.09554655 0.13127648 0.77317698]


In [ ]:
print("XGB:")
xgbm_preds = get_predictions(xgbm)

XGB:
Unconfident speech has fluency prediction High with max probability 0.9473029375076294 and all probabilities [0.0421029  0.01059419 0.94730294]
Confident speech has fluency prediction High with max probability 0.9822513461112976 and all probabilities [0.00461397 0.0131347  0.98225135]
Tim urban's speech has fluency prediction Medium with max probability 0.601862907409668 and all probabilities [0.39003375 0.6018629  0.00810337]
Almost silent speech has fluency prediction High with max probability 0.7871266007423401 and all probabilities [0.05669266 0.15618077 0.7871266 ]
Stuttering speech has fluency prediction High with max probability 0.6251384615898132 and all probabilities [0.05115107 0.32371047 0.62513846]


In [ ]:
print("Stack:")
preds = get_predictions(stacking)

Stack:
Unconfident speech has fluency prediction High with max probability 0.9443275032808817 and all probabilities [0.02133516 0.03433733 0.9443275 ]
Confident speech has fluency prediction High with max probability 0.9402191332266563 and all probabilities [0.02280849 0.03697238 0.94021913]
Tim urban's speech has fluency prediction Medium with max probability 0.5272438773992747 and all probabilities [0.43208016 0.52724388 0.04067596]
Almost silent speech has fluency prediction High with max probability 0.9163022030722637 and all probabilities [0.03414901 0.04954879 0.9163022 ]
Stuttering speech has fluency prediction High with max probability 0.9199538601446233 and all probabilities [0.03073385 0.04931229 0.91995386]


In [ ]:
print("Vote:")
preds = get_predictions(voting)

Vote:
Unconfident speech has fluency prediction Low with max probability 2 and all probabilities 2
Confident speech has fluency prediction Low with max probability 2 and all probabilities 2
Tim urban's speech has fluency prediction Low with max probability 0 and all probabilities 0
Almost silent speech has fluency prediction Low with max probability 2 and all probabilities 2
Stuttering speech has fluency prediction Low with max probability 2 and all probabilities 2


In [ ]:
print("Meta Forest:")
lgbm_preds_inputs = lgbm_preds.argmax(axis=1)
xgbm_preds_inputs = xgbm_preds.argmax(axis=1)

rf_inputs = np.array([[lgbm_pred, xgbm_pred] for lgbm_pred, xgbm_pred in zip(lgbm_preds_inputs, xgbm_preds_inputs)])

res = rf.predict_proba(rf_inputs)

speeches = ['Unconfident speech', "Confident speech", "Tim urban's speech", "Almost silent speech", "Stuttering speech"]
classes = ["Low", "Medium", "High"]
for i in range(len(res)):
    print(f"{speeches[i]} has fluency prediction {classes[res[i].argmax()]} with max probability {res[i].max()} and all probabilities {res[i]}")

Meta Forest:
Unconfident speech has fluency prediction High with max probability 0.9010900423579745 and all probabilities [0.03952113 0.05938883 0.90109004]
Confident speech has fluency prediction High with max probability 0.9010900423579745 and all probabilities [0.03952113 0.05938883 0.90109004]
Tim urban's speech has fluency prediction Medium with max probability 0.5211616479915353 and all probabilities [0.44664939 0.52116165 0.03218896]
Almost silent speech has fluency prediction High with max probability 0.9010900423579745 and all probabilities [0.03952113 0.05938883 0.90109004]
Stuttering speech has fluency prediction High with max probability 0.9010900423579745 and all probabilities [0.03952113 0.05938883 0.90109004]
